In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

/Users/brianconnor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [6]:
!pip install talos

In [7]:
# I looked around online for methods on hyperparameter tuning and found one where someone built their own
# gridsearch-like package to use specifically with keras... so here we go

In [2]:
import talos as talos

In [3]:
def hypertuner(Xtrain, ytrain, Xtest, ytest, params):
    
    network = Sequential()
    
    # input layer
    network.add(Dense(units=Xtrain.shape[1],
                      activation="relu",
                      input_dim=Xtrain.shape[1],
                      kernel_regularizer=params["regularizer"](params["lambda"])))
    
    network.add(Dropout(params["Dropout"]))
    
    # first optional hidden layer
    if params["hidden_layers"] >= 1:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"](params["lambda"])))
        
        network.add(Dropout(params["Dropout"]))
        
    # second optional hidden layer
    if params["hidden_layers"] >= 2:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"](params["lambda"])))
        
        network.add(Dropout(params["Dropout"]))    

    # third optional hidden layer    
    if params["hidden_layers"] >= 3:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"](params["lambda"])))
        
        network.add(Dropout(params["Dropout"]))

    # output layer
    network.add(Dense(units=1,
                      activation=None,
                      kernel_regularizer=params["regularizer"](params["lambda"])))
    
    # compiling
    network.compile(loss="mean_absolute_error", optimizer="adam")
    
    # fitting
    fitted_model = network.fit(Xtrain, 
                          ytrain, 
                          validation_data=[Xtest, ytest], 
                          epochs=params["epochs"],
                          verbose=0)
    
    return fitted_model, network

In [4]:
parameters = {
    "regularizer" : [regularizers.l1, regularizers.l2],
    "lambda" : [.005, .01, .05, .1],
    "Dropout" : [.1, .25, .5],
    "hidden_layers" : [0, 1, 2, 3],
    "hidden_nodes" : [50, 100, 150, 200],
    "epochs" : [300, 500, 1000]
}

In [5]:
train = pd.read_csv("./data/cleaned_train.csv")

In [6]:
train.drop(["Unnamed: 0", "week_start_date"], axis=1, inplace=True)

In [7]:
train = pd.get_dummies(train)

In [8]:
target = "total_cases"
features = [x for x in train if x != target]
y = train[target]
X = train[features]
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [9]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [10]:
y.values

array([4, 5, 4, ..., 1, 1, 4])

In [11]:
X

array([[-2.12857604, -0.56320208, -0.14019201, ..., -0.51225296,
        -0.83304245,  0.83304245],
       [-2.12857604, -0.49604779,  0.1965216 , ..., -0.66485238,
        -0.83304245,  0.83304245],
       [-2.12857604, -0.4288935 , -0.78336483, ...,  0.01153424,
        -0.83304245,  0.83304245],
       ...,
       [ 1.60779361, -0.22743062,  0.74577837, ..., -0.68959823,
         1.20041901, -1.20041901],
       [ 1.60779361, -0.16027633,  1.36408706, ..., -0.51225296,
         1.20041901, -1.20041901],
       [ 1.60779361, -0.09312204,  1.10974656, ..., -0.42151817,
         1.20041901, -1.20041901]])

In [ ]:
search = talos.Scan(x=X,
                    y=y.values,
                    params=parameters,
                    model=hypertuner,
                    dataset_name="whatthehell")

In [ ]:
search_1 = talos.Reporting("./train_.csv")

In [ ]:
search_1.data.sort_values("hidden_layers",ascending=False).head(10)

##### second run through

In [31]:
def hypertuner(Xtrain, ytrain, Xtest, ytest, params):
    
    network = Sequential()
    
    # input layer
    network.add(Dense(units=Xtrain.shape[1],
                      activation="relu",
                      input_dim=Xtrain.shape[1],
                      kernel_regularizer=params["regularizer"]))
    
    network.add(Dropout(params["Dropout"]))
    
    # first optional hidden layer
    if params["hidden_layers"] >= 1:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))
        
    # second optional hidden layer
    if params["hidden_layers"] >= 2:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))    

    # output layer
    network.add(Dense(units=1,
                      activation=None,
                      kernel_regularizer=params["regularizer"]))
    
    # compiling
    network.compile(loss="mean_absolute_error", optimizer="adam")
    
    early = EarlyStopping(monitor="val_loss", min_delta=0, patience=2)
    
    # fitting
    HISTORY = network.fit(Xtrain, 
                          ytrain, 
                          validation_data=[Xtest, ytest], 
                          epochs=params["epochs"],
                          callbacks=[early],
                          verbose=0)
    
    return HISTORY, network

In [32]:
parameters = {
    "regularizer" : [regularizers.l2(.001), regularizers.l2(.005), regularizers.l2(.01)],
#     "lambda" : [.001, .005, .01],
    "Dropout" : [.15, .25],
    "hidden_layers" : [0, 1, 2],
    "hidden_nodes" : [75, 150, 200],
    "epochs" : [500, 1000]
#     "min_delta" : [.01, .05, .1],
#     "patience" : [5,10,20]
}

In [33]:
search = talos.Scan(x=X,
                    y=y.values,
                    params=parameters,
                    model=hypertuner,
                    dataset_name="with_earlystop")





  0%|          | 0/108 [00:00<?, ?it/s]



  1%|          | 1/108 [00:02<05:14,  2.94s/it]



  2%|▏         | 2/108 [00:05<05:08,  2.91s/it]



  3%|▎         | 3/108 [00:08<05:06,  2.92s/it]



  4%|▎         | 4/108 [00:11<05:02,  2.90s/it]



  5%|▍         | 5/108 [00:15<05:18,  3.09s/it]



  6%|▌         | 6/108 [00:18<05:14,  3.09s/it]



  6%|▋         | 7/108 [00:20<05:01,  2.98s/it]



  7%|▋         | 8/108 [00:23<04:54,  2.94s/it]



  8%|▊         | 9/108 [00:26<04:40,  2.83s/it]



  9%|▉         | 10/108 [00:28<04:31,  2.77s/it]



 10%|█         | 11/108 [00:31<04:17,  2.66s/it]



 11%|█         | 12/108 [00:34<04:24,  2.75s/it]



 12%|█▏        | 13/108 [00:36<04:07,  2.60s/it]



 13%|█▎        | 14/108 [00:39<04:10,  2.66s/it]



 14%|█▍        | 15/108 [00:42<04:28,  2.89s/it]



 15%|█▍        | 16/108 [00:45<04:23,  2.87s/it]



 16%|█▌        | 17/108 [00:48<04:24,  2.90s/it]



 17%|█▋        | 18/108 [00:52<04:36,  3.07s/it]



 18%|█▊        | 19/108 [0

Scan Finished!


In [36]:
search.data.sort_values("val_loss", ascending=True).head()

,round_epochs,val_loss,loss,regularizer,Dropout,hidden_layers,hidden_nodes,epochs
70,35,11.697192295245571,13.196120418211612,<keras.regularizers.L1L2 object at 0x1a2bf693c8>,0.15,1,150,1000
20,46,11.77439817212188,13.473891176696812,<keras.regularizers.L1L2 object at 0x1a2bf693c8>,0.25,1,200,1000
14,16,11.83295660849297,13.623923868717181,<keras.regularizers.L1L2 object at 0x1a2bf693c8>,0.15,2,150,1000
56,29,11.845305246539356,13.588789221114508,<keras.regularizers.L1L2 object at 0x1a2bf693c8>,0.15,1,75,1000
17,23,11.882311800853557,13.386529070334689,<keras.regularizers.L1L2 object at 0x1a2bf693c8>,0.15,2,75,500


##### third runthrough with polynomial weather

In [41]:
train = pd.read_csv("./data/cleaned_train.csv")
train.drop(["Unnamed: 0", "week_start_date"], axis=1, inplace=True)
train = pd.get_dummies(train)

non_weather = "year weekofyear ndvi_ne ndvi_nw ndvi_se ndvi_sw total_cases city_sj city_iq".split()
weather_features = [x for x in train.columns if x not in non_weather]

pf = PolynomialFeatures(degree=2, include_bias=False)
pf_feat = pd.DataFrame(data=pf.fit_transform(train[weather_features]), columns=pf.get_feature_names(weather_features))

train_weather = train.copy()
train_weather.drop(labels=weather_features, axis=1, inplace=True)
train_weather = train_weather.join(pf_feat, how="outer")

target = "total_cases"
features = [x for x in train_weather if x != target]
y = train_weather[target]
X = train_weather[features]

ss = StandardScaler()
X = ss.fit_transform(X)


In [42]:
def hypertuner(Xtrain, ytrain, Xtest, ytest, params):
    
    network = Sequential()
    
    # input layer
    network.add(Dense(units=Xtrain.shape[1],
                      activation="relu",
                      input_dim=Xtrain.shape[1],
                      kernel_regularizer=params["regularizer"]))
    
    network.add(Dropout(params["Dropout"]))
    
    # first optional hidden layer
    if params["hidden_layers"] >= 1:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))
        
    # second optional hidden layer
    if params["hidden_layers"] >= 2:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))    

    # output layer
    network.add(Dense(units=1,
                      activation=None,
                      kernel_regularizer=params["regularizer"]))
    
    # compiling
    network.compile(loss="mean_absolute_error", optimizer="adam")
    
    early = EarlyStopping(monitor="val_loss", min_delta=0, patience=params["patience"])
    
    # fitting
    HISTORY = network.fit(Xtrain, 
                          ytrain, 
                          validation_data=[Xtest, ytest], 
                          epochs=params["epochs"],
                          callbacks=[early],
                          verbose=0)
    
    return HISTORY, network

In [43]:
parameters = {
    "regularizer" : [regularizers.l2(.001), regularizers.l2(.005), regularizers.l2(.01)],
#     "lambda" : [.001, .005, .01],
    "Dropout" : [.15, .25],
    "hidden_layers" : [0, 1, 2],
    "hidden_nodes" : [75, 150, 200],
    "epochs" : [500, 1000],
#     "min_delta" : [.01, .05, .1],
    "patience" : [3,5,10]
}

In [44]:
search_3 = talos.Scan(x=X,
                    y=y.values,
                    params=parameters,
                    model=hypertuner,
                    dataset_name="with_earlystop")





  0%|          | 0/324 [00:00<?, ?it/s]



  0%|          | 1/324 [00:02<13:32,  2.51s/it]



  1%|          | 2/324 [00:05<13:51,  2.58s/it]



  1%|          | 3/324 [00:08<14:52,  2.78s/it]



  1%|          | 4/324 [00:10<13:57,  2.62s/it]



  2%|▏         | 5/324 [00:13<14:33,  2.74s/it]



  2%|▏         | 6/324 [00:16<14:40,  2.77s/it]



  2%|▏         | 7/324 [00:21<18:02,  3.41s/it]



  2%|▏         | 8/324 [00:25<19:21,  3.68s/it]



  3%|▎         | 9/324 [00:29<19:12,  3.66s/it]



  3%|▎         | 10/324 [00:32<17:34,  3.36s/it]



  3%|▎         | 11/324 [00:34<16:27,  3.16s/it]



  4%|▎         | 12/324 [00:36<14:51,  2.86s/it]



  4%|▍         | 13/324 [00:40<15:23,  2.97s/it]



  4%|▍         | 14/324 [00:43<15:59,  3.10s/it]



  5%|▍         | 15/324 [00:48<18:11,  3.53s/it]



  5%|▍         | 16/324 [00:50<17:04,  3.32s/it]



  5%|▌         | 17/324 [00:55<18:52,  3.69s/it]



  6%|▌         | 18/324 [00:57<16:46,  3.29s/it]



  6%|▌         | 19/324 [0

 78%|███████▊  | 253/324 [13:13<03:39,  3.09s/it]



 78%|███████▊  | 254/324 [13:16<03:25,  2.93s/it]



 79%|███████▊  | 255/324 [13:19<03:31,  3.07s/it]



 79%|███████▉  | 256/324 [13:22<03:26,  3.04s/it]



 79%|███████▉  | 257/324 [13:25<03:25,  3.07s/it]



 80%|███████▉  | 258/324 [13:27<03:04,  2.80s/it]



 80%|███████▉  | 259/324 [13:29<02:46,  2.56s/it]



 80%|████████  | 260/324 [13:32<02:48,  2.63s/it]



 81%|████████  | 261/324 [13:35<02:51,  2.72s/it]



 81%|████████  | 262/324 [13:39<03:08,  3.05s/it]



 81%|████████  | 263/324 [13:42<03:11,  3.13s/it]



 81%|████████▏ | 264/324 [13:47<03:33,  3.56s/it]



 82%|████████▏ | 265/324 [13:50<03:28,  3.53s/it]



 82%|████████▏ | 266/324 [13:52<03:00,  3.12s/it]



 82%|████████▏ | 267/324 [13:56<03:00,  3.16s/it]



 83%|████████▎ | 268/324 [13:59<03:05,  3.31s/it]



 83%|████████▎ | 269/324 [14:03<03:06,  3.39s/it]



 83%|████████▎ | 270/324 [14:05<02:44,  3.05s/it]



 84%|████████▎ | 271/324 [14:07<02:28,  2.80s/

Scan Finished!


In [45]:
search_3.data.sort_values("val_loss", ascending=True).head(10)

,round_epochs,val_loss,loss,regularizer,Dropout,hidden_layers,hidden_nodes,epochs,patience
38,39,13.14453761118383,12.735324829378053,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,2,75,1000,10
144,55,13.187607075734,11.610463613291964,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,1,150,500,10
48,47,13.195705690610378,12.439977362903841,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.25,2,150,1000,10
300,42,13.209139587382213,11.671812720638979,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,2,200,1000,10
101,43,13.21966235933329,12.209987312049044,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,1,75,500,5
166,47,13.251895285533411,12.362597378031989,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,1,75,1000,10
121,54,13.269675773177738,12.265539368038674,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.25,2,75,1000,10
321,64,13.272631621297872,11.628183586276132,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.15,1,200,500,10
274,29,13.281848527510752,13.179252612631291,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.25,2,75,1000,5
191,39,13.28947878512984,12.755266101012008,<keras.regularizers.L1L2 object at 0x1a2c8a1198>,0.25,2,200,1000,10


#### fourth runthrough with whatever, none of these are scoring better than the others

In [14]:
def hypertuner(Xtrain, ytrain, Xtest, ytest, params):
    
    network = Sequential()
    
    # input layer
    network.add(Dense(units=Xtrain.shape[1],
                      activation="relu",
                      input_dim=Xtrain.shape[1],
                      kernel_regularizer=params["regularizer"]))
    
    network.add(Dropout(params["Dropout"]))
    
    # first optional hidden layer
    if params["hidden_layers"] >= 1:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))
        
    # second optional hidden layer
    if params["hidden_layers"] >= 2:
        network.add(Dense(units=params["hidden_nodes"],
                          activation="relu",
                          kernel_regularizer=params["regularizer"]))
        
        network.add(Dropout(params["Dropout"]))    

    # output layer
    network.add(Dense(units=1,
                      activation=None,
                      kernel_regularizer=params["regularizer"]))
    
    # compiling
    network.compile(loss="mean_absolute_error", optimizer="adam")
    
    early = EarlyStopping(monitor="val_loss", min_delta=0, patience=params["patience"])
    
    # fitting
    HISTORY = network.fit(Xtrain, 
                          ytrain, 
                          validation_data=[Xtest, ytest], 
                          epochs=params["epochs"],
                          callbacks=[early],
                          verbose=0)
    
    return HISTORY, network

In [15]:
parameters = {
    "regularizer" : [regularizers.l1(.001), regularizers.l1(.005), regularizers.l1(.01)],
#     "lambda" : [.001, .005, .01],
    "Dropout" : [.10, .20, .30],
    "hidden_layers" : [0, 1, 2],
    "hidden_nodes" : [75, 150, 200],
    "epochs" : [1000],
#     "min_delta" : [.01, .05, .1],
    "patience" : [3,5,10]
}

In [16]:
search = talos.Scan(x=X,
                    y=y.values,
                    params=parameters,
                    model=hypertuner,
                    dataset_name="l1_early")

100%|██████████| 243/243 [19:43<00:00,  4.53s/it]

Scan Finished!


In [17]:
search.data.sort_values("val_loss", ascending=True).head(10)

,round_epochs,val_loss,loss,regularizer,Dropout,hidden_layers,hidden_nodes,epochs,patience
62,134,12.61628206298345,12.2592851362304,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.2,1,200,1000,10
180,129,12.646182854131533,12.287859492231739,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.1,1,75,1000,10
182,101,12.766758747654414,12.538079038211741,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.2,1,150,1000,10
15,117,12.770896727931845,12.849036694113018,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.3,1,75,1000,10
30,87,12.772199616897703,12.520099592370869,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.1,1,75,1000,5
139,117,12.777913253351377,12.399868813943485,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.1,1,150,1000,10
140,302,12.783902663983268,12.436534545699711,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.1,0,200,1000,10
86,97,12.793992486666879,13.024692898293448,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.3,1,75,1000,5
43,341,12.810642317920372,12.703599601477755,<keras.regularizers.L1L2 object at 0x1a218b7898>,0.1,0,150,1000,10
101,176,12.828607919033725,12.946868978567329,<keras.regularizers.L1L2 object at 0x1a218b79e8>,0.3,0,150,1000,10
